In [1]:
!pip install transformers==4.28.0 evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2


In [2]:
text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."

In [3]:
from transformers import pipeline

translator = pipeline("translation_en_to_fr", model="ethansimrm/t5_small_prelim_emea_enfr")
translator(text)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

[{'translation_text': 'Les légumes partagent les ressources avec les bactéries fixatrices de l’ azote.'}]

In [4]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("HF_KEY")
login(secret_value)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
from datasets import load_dataset
wmt16_test = load_dataset("ethansimrm/wmt16_biomed_test", use_auth_token=True)
wmt16_gold = load_dataset("ethansimrm/wmt16_biomed_gold", use_auth_token=True)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/ethansimrm--wmt16_biomed_test-195b3b7168424280/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/ethansimrm--wmt16_biomed_gold-4a81ce75a76006f6/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
source_sentences = wmt16_test["test"]["passage/0/sentence/0/text"]
target_sentences = wmt16_gold["train"]["passage/0/sentence/0/text"]

In [7]:
import evaluate
metric = evaluate.load("sacrebleu")

In [8]:
prefixed_ss = ["translate English to French: " + s for s in source_sentences]

In [9]:
from tqdm import tqdm
tqdm._instances.clear() #Stops newline printing error
predictions = [translator(s) for s in tqdm(prefixed_ss)]

100%|██████████| 500/500 [04:21<00:00,  1.91it/s]


In [10]:
preds = []
for p in predictions:
    preds.append(p[0]['translation_text'])

In [11]:
result = metric.compute(predictions=preds, references=target_sentences) #BLEU score for provided input and references
result = {"bleu": result["score"]}

In [12]:
result

{'bleu': 31.241005725769455}